## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Options,
    Session,
    Sampler,
    RuntimeOptions,
)
from quantum_serverless import QuantumServerless

from circuit_knitting_toolbox.circuit_cutting import WireCutter

In [2]:
serverless = QuantumServerless()

### Create a circuit to cut

In [3]:
num_qubits = 8
circuit = EfficientSU2(
    num_qubits=num_qubits,
    reps=2,
    entanglement="linear",
    su2_gates=["ry"],
    insert_barriers=False,
)

shift = 0.25
params = (
    [np.pi / 2 - shift]
    + [shift]
    + [np.pi / 2 - shift]
    + [shift] * int(len(circuit.parameters) / 1 - 3)
)

circuit.assign_parameters(params, inplace=True)
circuit = circuit.decompose()

### Set up the Qiskit runtime service

In [4]:
service_args = QiskitRuntimeService(
    channel="ibm_quantum",
    instance="<YOUR_INSTANCE>",
    token="<YOUR_API_KEY>",
).active_account()

### Find cuts that match our criteria

In [5]:
# Set the Sampler and runtime options
# options = Options(resilience_level=1, optimization_level=3, execution={"shots": 8192})
# runtime_options = RuntimeOptions(backend="ibmq_qasm_simulator")

# Instantiate a WireCutter and decompose the circuit
# cutter = WireCutter(
#    qc, service_args=service_args, backend_names=backend_names, options=options, runtime_options=runtime_options
# )
backend_names = ["ibmq_qasm_simulator", "ibmq_qasm_simulator"]
cutter = WireCutter(circuit, service_args=service_args, backend_names=backend_names)  # Local Estimator

with serverless:
    cuts = cutter.decompose(
        method="automatic",
        max_subcircuit_width=6,
        max_cuts=2,
        num_subcircuits=[2],
    )

(_cut_automatic pid=77720) Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
(_cut_automatic pid=77720) Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
(_cut_automatic pid=77720) CPXPARAM_Read_DataCheck                          1
(_cut_automatic pid=77720) CPXPARAM_TimeLimit                               300
(_cut_automatic pid=77720) Warning:  Non-integral bounds for integer variables rounded.
(_cut_automatic pid=77720) Tried aggregator 3 times.
(_cut_automatic pid=77720) MIP Presolve eliminated 37 rows and 8 columns.
(_cut_automatic pid=77720) MIP Presolve modified 7 coefficients.
(_cut_automatic pid=77720) Aggregator did 103 substitutions.
(_cut_automatic pid=77720) Reduced MIP has 366 rows, 127 columns, and 1072 nonzeros.
(_cut_automatic pid=77720) Reduced MIP has 121 binaries, 6 generals, 0 SOSs, and 0 indicators.
(_cut_automatic pid=77720) Presolve time = 0.00 sec. (2.11 ticks)
(_cut_automatic pid=77720) Found incumbent of value 2.0

/Users/caleb/opt/anaconda3/envs/summitdemo3py37/lib/python3.7/site-packages/ray/_private/worker.py:976: UserWarning: len(ctx) is deprecated. Use len(ctx.address_info) instead.
  warnings.warn("len(ctx) is deprecated. Use len(ctx.address_info) instead.")


### Evaluate the subcircuits, then recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [6]:
# Evaluate the subcircuits on backend
with serverless:
    subcircuit_instance_probabilities = cutter.evaluate(cuts)

# Recompose the circuit and generate the cut circuit's probability distribution
with serverless:
    reconstructed_probabilities = cutter.recompose(
        subcircuit_instance_probabilities, cuts, num_threads=4
    )

# Use a statevector simulator to calculate the error between the inferred and actual distributions
metrics = cutter.verify(reconstructed_probabilities)
print(metrics)

{'nearest': {'chi2': 0.01954538514207468, 'Mean Squared Error': 3.6903448954462296e-07, 'Mean Absolute Percentage Error': 2086.0110048217257, 'Cross Entropy': 2.756155743996132, 'HOP': 0.9924607197002497}, 'naive': {'chi2': 0.01834240070622006, 'Mean Squared Error': 3.3782562647441804e-07, 'Mean Absolute Percentage Error': 2585.081449491616, 'Cross Entropy': 2.734403698464808, 'HOP': 0.9908939973952848}}
